# Description

This exercise continues to work with the small "library" of functions on integers in the setup perform some simple number theoretic or combinatorics operations.  They are little documented and each has a somewhat dense implementation.  From the prior exercises, you will have thought some about relevant tests, and implemented them in doctest and unittest styles.

For this exercise, you should write a single `unittest` class called `TestLibrary2`.  A distinct name is used from that in the prior exercise merely because both should be independently valuable, and a real library will probably use multiple test classes.

In order to utilize some capabilities discussed in this lesson, your new tests should include:

* Consider primes in non-overlapping blocks of 20 at a time.  Calculate the median value of "reachable" numbers based on that block.  Here reachable means that a number can be obtained by summing some of the numbers from the block.  
  * Specifically, for prime numbers, this procedure will give you the values 29, 113, 229, 349, 463.  
  * **As separate subtests**, check each such value is obtained with the functions in the library.

* Test how function respond to improper arguments.  
  * What does `pair_sums()` do if passed a string? Write a test for that.  
  * What does `pair_sums()` do if passed an integer? Write a test for that.  
  * What about a list of strings? 
  * What about a heterogeneous list of numbers and strings? 
  * What about a list of numbers that includes the special value `nan`? E.g. `[1, 2, 3, nan]`.

Add any other tests you find useful, especially ones utilizing techniques you learned in this lesson.

The exercises in this courses have a large space of possible solutions.  The "solution" provided gives plausible tests to include, but yours may well be substantially different and yet no less useful or relevant.  Your solution might well be better than that suggested.

# Setup

In [19]:
import unittest
from math import sqrt, log, nan

def get_primes_upto(limit):
    "A list of all primes less than or equal to limit"
    is_prime = [False] * 2 + [True] * (limit-1)
    for n in range(int(sqrt(limit) + 1.5)): 
        if is_prime[n]:
            is_prime[n**2::n] = [False] * ((limit - n**2)//n + 1)
    return trues(is_prime)

def prime_count(limit):
    "Upper bound on number of primes below a limit"
    # Gauss/Legendre approx, padded to exceed π(x) for small limits
    return int(1.2 * limit/log(limit))

def get_init_primes(N):
    "Return the first N prime numbers"
    # Find "enough" primes
    limit = 8
    while N > prime_count(limit):
        limit *= 2
    many_primes = get_primes_upto(limit)
    # Return exactly N of them
    return many_primes[:N]

def trues(it): 
    "Which elements of 'bitfield iterable' are True?"
    return [n for n, target in enumerate(it) if target]

def sums_of_subset(numbers):
    "The natural numbers that are sums of subsets of initial set"
    numbers = sorted(numbers)                 # Numbers in ascending order
    sum_of_numbers = sum(numbers)
    reachable = [False] * (sum_of_numbers+1)  # Trues as one-based index
    for p in numbers:
        reachable[p] = True 
        for n, target in enumerate(reachable[:]):
            if target and  n != p:
                reachable[p+n] = True
    return trues(reachable)

def pair_sums(numbers, allow_doubles=False):
    "Sums of elements from initial set"
    sums = set()
    numbers = sorted(numbers)
    for i in numbers:
        for j in numbers:
            if allow_doubles or i != j:
                sums.add(i+j)
    return sums

# Solution

In [83]:
class TestLibrary2(unittest.TestCase):
    def setUp(self):
        self.primes100 = get_init_primes(100)
        
    def test_reachable_from_primes(self):
        from statistics import median_low
        medians = [29, 113, 229, 349, 463]
        for med, n in zip(medians, range(0, 100, 20)):
            with self.subTest(n=n):
                group = self.primes100[n:n+20]
                self.assertEqual(med, median_low(group))
    
    def test_pair_sum_string(self):
        pairs = pair_sums('abcd')
        self.assertIsInstance(pairs, set)
        self.assertTrue(len(pairs) == 12)
        for s in {'ab', 'ac', 'bd', 'db', 'dc'}:
            with self.subTest(s=s):
                self.assertIn(s, pairs)

    def test_pair_sum_int(self):
        with self.assertRaises(TypeError):
            pairs = pair_sums(1234)

    def test_pair_sum_string_list(self):
        pairs = pair_sums(['a', 'b', 'c', 'd'])
        self.assertIsInstance(pairs, set)
        self.assertTrue(len(pairs) == 12)
        for s in {'ab', 'ac', 'bd', 'db', 'dc'}:
            with self.subTest(s=s):
                self.assertIn(s, pairs)

    def test_pair_sum_mixed_list(self):
        with self.assertRaises(TypeError):
            pairs = pair_sums(['a', 'b', 'c', 1])

    def test_pair_sum_nans(self):
        # nans are unequal to themselves... hmm...
        from math import isnan
        pairs = pair_sums([1, 2, 3, nan])
        self.assertEqual(len(pairs), 10)
        self.assertEqual(sum(isnan(n) for n in pairs), 7)

# Test Cases

In [84]:
def test_is_test_class():
    assert issubclass(TestLibrary2, unittest.TestCase)
    
test_is_test_class()

In [85]:
def test_all():
    runner = TestLibrary2()
    runner.setUp()
    tests = [getattr(runner, name) for name in dir(runner) 
             if name.startswith('test_')]
    assert len(tests) >= 6, "At least 6 tests are required"
    for test in tests:
        test()

test_all()